# TensorFlow和Keras混合编程

In [1]:
# -*- coding: utf-8 -*-

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from six.moves import xrange

## 方式1：使用tf.keras的Sequential和Functional建模

In [3]:
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Sequential
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax),
])

# Functional
# input_tensor = tf.keras.layers.Input(shape=(28, 28,))
# inner = tf.keras.layers.Flatten()(input_tensor)
# inner = tf.keras.layers.Dense(128, activation=tf.nn.relu)(inner)
# output_tensor = tf.keras.layers.Dense(10, activation=tf.nn.softmax)(inner)
# model = tf.keras.models.Model(input_tensor, output_tensor)

model.compile(optimizer=tf.train.AdamOptimizer(),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=10000, epochs=1)
result = model.evaluate(x_test, y_test)
print(result)

Epoch 1/1
10000/10000 [==============================] - 0s 42us/step
[1.57093619556427, 0.6804]


## 方式2：使用tf.keras.layers和Functional代码代替tf.nn.layers

In [4]:
x = tf.placeholder(dtype=tf.float32, shape=[None, 28, 28], name='placeholder_x')
y = tf.placeholder(dtype=tf.int64, shape=[None, ], name='placeholder_y')

# Functional
inner = tf.keras.layers.Flatten()(x)
inner = tf.keras.layers.Dense(128, activation=tf.nn.relu)(inner)
output_tensor = tf.keras.layers.Dense(10)(inner)

loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=output_tensor))
optimizer_op = tf.train.AdamOptimizer().minimize(loss_op)
accuracy_op = tf.reduce_mean(tf.cast(tf.equal(y, tf.argmax(output_tensor, axis=-1)), dtype=tf.float32))

batch_size = 10000
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for _ in xrange(1):
    for i in xrange(len(x_train) // batch_size):
        batch_x, batch_y = x_train[i * batch_size:(i + 1) * batch_size], y_train[i * batch_size:(i + 1) * batch_size]
        _, batch_loss, batch_acc = sess.run([optimizer_op, loss_op, accuracy_op], feed_dict={x: batch_x, y: batch_y})
        print('loss: %-.5f, acc: %-.5f' % (batch_loss, batch_acc))

loss: 2.35689, acc: 0.12800
loss: 2.19394, acc: 0.22910
loss: 2.04874, acc: 0.36950
loss: 1.94520, acc: 0.45700
loss: 1.82590, acc: 0.54090
loss: 1.68981, acc: 0.62480
